In [1]:
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt

pd.options.mode.chained_assignment = None

riskdf = pd.read_sas('Resources\LLCP2013.XPT')

fip_state = pd.read_csv('fips_to_state.csv')

riskdf.head()

,_STATE,FMONTH,IDATE,IMONTH,IDAY,IYEAR,DISPCODE,SEQNO,_PSU,CTELENUM,...,_AGE80,_IMPEDUC,_IMPMRTL,_IMPHOME,RCSBRAC1,RCSRACE1,RCHISLA1,RCSBIRTH,TYPEINDS,TYPEWORK
0,1.0,1.0,b'01092013',b'01',b'09',b'2013',1100.0,2.013001e+09,2.013001e+09,1.0,...,60.0,6.0,2.0,1.0,NaN,b'',b'',b'',b'',b''
1,1.0,1.0,b'01192013',b'01',b'19',b'2013',1100.0,2.013001e+09,2.013001e+09,1.0,...,50.0,5.0,1.0,1.0,NaN,b'',b'',b'',b'',b''
2,1.0,1.0,b'01192013',b'01',b'19',b'2013',1100.0,2.013001e+09,2.013001e+09,1.0,...,55.0,6.0,1.0,1.0,NaN,b'',b'',b'',b'',b''
3,1.0,1.0,b'01112013',b'01',b'11',b'2013',1100.0,2.013001e+09,2.013001e+09,1.0,...,64.0,4.0,1.0,1.0,NaN,b'',b'',b'',b'',b''
4,1.0,2.0,b'02062013',b'02',b'06',b'2013',1100.0,2.013001e+09,2.013001e+09,1.0,...,66.0,6.0,1.0,1.0,NaN,b'',b'',b'',b'',b''


In [2]:
# Re-naming the _STATE to fips for merging the FIPS DF and New DF# Re-nam 
riskdf = riskdf.rename(columns={"_STATE":"fips"})
riskdf = riskdf.merge(fip_state,on="fips")
# Dropping the fips column as it is not required further
riskdf.drop(["fips"],axis=1,inplace=True)

In [3]:
#create a new dataframe with the columns we want
newdf = riskdf[['state_abbr', 'IDATE' , '_BMI5CAT', '_RFBMI5', '_RACE', '_RACEGR3']]

newdf.head()

,state_abbr,IDATE,_BMI5CAT,_RFBMI5,_RACE,_RACEGR3
0,AL,b'01092013',4.0,2.0,2.0,2.0
1,AL,b'01192013',1.0,1.0,1.0,1.0
2,AL,b'01192013',3.0,2.0,1.0,1.0
3,AL,b'01112013',2.0,1.0,1.0,1.0
4,AL,b'02062013',4.0,2.0,1.0,1.0


In [4]:
#fill in columns with data based on the 2016 codebook
#reference: https://www.cdc.gov/brfss/annual_data/2016/pdf/codebook16_llcp.pdf
newdf["IDATE"] = newdf["IDATE"].str.decode("utf-8")
newdf["_BMI5CAT"] = newdf["_BMI5CAT"].replace({1:"Underweight", 2.0:"Normal Weight", 3.0:"Overweight", 4.0:"Obese", None:"Don't know/Refused/Missing"})
newdf["_RFBMI5"]= newdf["_RFBMI5"].replace({1.0:"No", 2.0:"Yes", 9.0:"Don’t know/Refused/Missing"})
newdf["_RACE"] = newdf["_RACE"].replace({1.0:"White, Non-Hispanic", 2.0:"Black, Non-Hispanic", 3.0:"American Indian/Alaskan Native", 4.0:"Asian", 5.0: "Native Hawaiian/Pacific Islander", 6.0: "Other Race, Non-Hispanic", 7.0: "Multiracial, Non-Hispanic", 8.0: "Hispanic", 9.0: "Don't Know/Not Sure/Refused"})

newdf = newdf.rename(columns={"state_abbr": "State", 
                              "IDATE": "Date", 
                              "_BMI5CAT": "BMI Category", 
                              "_RFBMI5": "BMI Over 25", 
                              "_RACE": "Race/Ethnicity (General)",})

newdf.head()

,State,Date,BMI Category,BMI Over 25,Race/Ethnicity (General),_RACEGR3
0,AL,01092013,Obese,Yes,"Black, Non-Hispanic",2.0
1,AL,01192013,Underweight,No,"White, Non-Hispanic",1.0
2,AL,01192013,Overweight,Yes,"White, Non-Hispanic",1.0
3,AL,01112013,Normal Weight,No,"White, Non-Hispanic",1.0
4,AL,02062013,Obese,Yes,"White, Non-Hispanic",1.0


In [5]:
#save to csv
newdf.to_csv("ObesityEthnicity2013.csv",index=False)